<a href="https://colab.research.google.com/github/annisatwain/AnalisisSentimen/blob/main/revisiiiii.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


install library yang dibutuhkan

In [4]:
!pip install nltk scikit-learn gensim pyLDAvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 72.1 MB/s eta 0:00:00


In [5]:
import pandas as pd
from sklearn.metrics import confusion_matrix, f1_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [6]:
import nltk
import re
import gensim
from gensim import corpora
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models.ldamodel import LdaModel
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from gensim.models import Word2Vec
import numpy as np

In [7]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

understanding data

In [8]:
data = pd.read_csv('/content/drive/MyDrive/Dataset /data_real.csv')

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


data yang ditampilkan berasal dari dataset sebelumnya dengan label komen negatif dan komen positif


In [9]:
data

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0.1,Unnamed: 0,Ulasan,Rating,Kategori,Nama Produk,Id Produk,Terjual,Id_Toko,Url,label
0,0,59,Pesanan gx sesuai dgn yg d gambar,2,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,https://www.tokopedia.com/timurjaya46/isi-stap...,0
1,1,70,Pengiriman dan respon super cepat. Tpi barang ...,2,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,https://www.tokopedia.com/timurjaya46/isi-stap...,0
2,2,95,barang yg dikirim tidak sesuai pesanan,1,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,https://www.tokopedia.com/timurjaya46/isi-stap...,0
3,3,117,"Php, bilang isi ada diseskripsi pas dipesen be...",1,pertukangan,steples tembak / staples tembak kenmaster PROM...,88842566,45,1102298,https://www.tokopedia.com/cahayabelawa/steples...,0
4,4,153,barang jelek copot&#34; terus tmpat taro isinya,2,pertukangan,Staple Gun / Staples Tembak / Staples Jok / He...,209226141,171,580197,https://www.tokopedia.com/hmhhardware/staple-g...,0
...,...,...,...,...,...,...,...,...,...,...,...
1920,995,1017,Bagus dan memuaskan. Packing bagus cepat,4,pertukangan,Staples Gun Tacker Mollar 3 in 1 / Staple Jok ...,221298191,408,1114588,https://www.tokopedia.com/indahjayatools/stapl...,1
1921,996,1018,Mantap pengiriman ok cepat recomended,5,pertukangan,Staples Gun Tacker Mollar 3 in 1 / Staple Jok ...,221298191,408,1114588,https://www.tokopedia.com/indahjayatools/stapl...,1
1922,997,1019,Pengiriman cepat. Lumayan,4,pertukangan,Staples Gun Tacker Mollar 3 in 1 / Staple Jok ...,221298191,408,1114588,https://www.tokopedia.com/indahjayatools/stapl...,1
1923,998,1020,belom dicoba semoga oke barangnya,4,pertukangan,Staples Gun Tacker Mollar 3 in 1 / Staple Jok ...,221298191,408,1114588,https://www.tokopedia.com/indahjayatools/stapl...,1


In [10]:
data_new = data.copy()

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


disini saya mencoba untuk membersihkan kolom yang tidak diperlukan

In [11]:
data_new.drop(['Unnamed: 0.1', 'Url', 'Unnamed: 0', 'label'], axis=1, inplace=True)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
data_new

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Ulasan,Rating,Kategori,Nama Produk,Id Produk,Terjual,Id_Toko
0,Pesanan gx sesuai dgn yg d gambar,2,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393
1,Pengiriman dan respon super cepat. Tpi barang ...,2,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393
2,barang yg dikirim tidak sesuai pesanan,1,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393
3,"Php, bilang isi ada diseskripsi pas dipesen be...",1,pertukangan,steples tembak / staples tembak kenmaster PROM...,88842566,45,1102298
4,barang jelek copot&#34; terus tmpat taro isinya,2,pertukangan,Staple Gun / Staples Tembak / Staples Jok / He...,209226141,171,580197
...,...,...,...,...,...,...,...
1920,Bagus dan memuaskan. Packing bagus cepat,4,pertukangan,Staples Gun Tacker Mollar 3 in 1 / Staple Jok ...,221298191,408,1114588
1921,Mantap pengiriman ok cepat recomended,5,pertukangan,Staples Gun Tacker Mollar 3 in 1 / Staple Jok ...,221298191,408,1114588
1922,Pengiriman cepat. Lumayan,4,pertukangan,Staples Gun Tacker Mollar 3 in 1 / Staple Jok ...,221298191,408,1114588
1923,belom dicoba semoga oke barangnya,4,pertukangan,Staples Gun Tacker Mollar 3 in 1 / Staple Jok ...,221298191,408,1114588


text prepocessing

In [13]:
reviews = data_new['Ulasan']

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('indonesian'))
lemmatizer = WordNetLemmatizer()

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    text = ' '.join([word for word in text.split() if word not in stop_words])
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    return text

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
data_new['Ulasan_Bersih'] = reviews.apply(preprocess_text)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
data_new

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Ulasan,Rating,Kategori,Nama Produk,Id Produk,Terjual,Id_Toko,Ulasan_Bersih
0,Pesanan gx sesuai dgn yg d gambar,2,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,pesanan gx sesuai dgn yg d gambar
1,Pengiriman dan respon super cepat. Tpi barang ...,2,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,pengiriman respon super cepat tpi barang sesua...
2,barang yg dikirim tidak sesuai pesanan,1,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,barang yg dikirim sesuai pesanan
3,"Php, bilang isi ada diseskripsi pas dipesen be...",1,pertukangan,steples tembak / staples tembak kenmaster PROM...,88842566,45,1102298,php bilang isi diseskripsi pa dipesen berbeda ...
4,barang jelek copot&#34; terus tmpat taro isinya,2,pertukangan,Staple Gun / Staples Tembak / Staples Jok / He...,209226141,171,580197,barang jelek copot tmpat taro isinya
...,...,...,...,...,...,...,...,...
1920,Bagus dan memuaskan. Packing bagus cepat,4,pertukangan,Staples Gun Tacker Mollar 3 in 1 / Staple Jok ...,221298191,408,1114588,bagus memuaskan packing bagus cepat
1921,Mantap pengiriman ok cepat recomended,5,pertukangan,Staples Gun Tacker Mollar 3 in 1 / Staple Jok ...,221298191,408,1114588,mantap pengiriman ok cepat recomended
1922,Pengiriman cepat. Lumayan,4,pertukangan,Staples Gun Tacker Mollar 3 in 1 / Staple Jok ...,221298191,408,1114588,pengiriman cepat lumayan
1923,belom dicoba semoga oke barangnya,4,pertukangan,Staples Gun Tacker Mollar 3 in 1 / Staple Jok ...,221298191,408,1114588,belom dicoba semoga oke barangnya


In [18]:
cleaned_reviews = data_new['Ulasan_Bersih']

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


embedding

In [19]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(cleaned_reviews.tolist())

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
data_new['embeddings'] = embeddings.tolist()

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
tokenized_reviews = [review.split() for review in cleaned_reviews]

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


pelabelan menggunakan LDA

In [22]:
dictionary = corpora.Dictionary(tokenized_reviews)
corpus = [dictionary.doc2bow(comment) for comment in tokenized_reviews]

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=2, passes=10)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
topics = lda_model.print_topics(num_words=5)
print("Topik yang ditemukan oleh LDA:")
for idx, topic in topics:
    print(f"Topik {idx}: {topic}")

lda_display = gensimvis.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(lda_display)


def assign_topic(lda_model, comment):
    bow = dictionary.doc2bow(comment)
    topic_distribution = lda_model.get_document_topics(bow)
    return max(topic_distribution, key=lambda x: x[1])[0]

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Topik yang ditemukan oleh LDA:
Topik 0: 0.030*"barang" + 0.018*"yg" + 0.014*"ga" + 0.012*"gan" + 0.012*"nya"
Topik 1: 0.054*"barang" + 0.047*"sesuai" + 0.019*"bagus" + 0.017*"cepat" + 0.015*"terima"


In [25]:
data_new['topic'] = [assign_topic(lda_model, comment) for comment in tokenized_reviews]

data_new['komentar'] = data_new['topic'].apply(lambda x: 'bot' if x == 0 else 'human')

# Menampilkan dataset yang sudah dilabeli
print("\nData dengan label:")
print(data_new[['Ulasan_Bersih', 'komentar', 'topic']].head())

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Data dengan label:
                                       Ulasan_Bersih komentar  topic
0                  pesanan gx sesuai dgn yg d gambar    human      1
1  pengiriman respon super cepat tpi barang sesua...    human      1
2                   barang yg dikirim sesuai pesanan    human      1
3  php bilang isi diseskripsi pa dipesen berbeda ...      bot      0
4               barang jelek copot tmpat taro isinya    human      1


In [26]:
data_new

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Ulasan,Rating,Kategori,Nama Produk,Id Produk,Terjual,Id_Toko,Ulasan_Bersih,embeddings,topic,komentar
0,Pesanan gx sesuai dgn yg d gambar,2,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,pesanan gx sesuai dgn yg d gambar,"[-0.0696227177977562, 0.011958673596382141, -0...",1,human
1,Pengiriman dan respon super cepat. Tpi barang ...,2,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,pengiriman respon super cepat tpi barang sesua...,"[-0.05312284454703331, 0.10111098736524582, -0...",1,human
2,barang yg dikirim tidak sesuai pesanan,1,pertukangan,Isi Staples Tembak 8 mm best guard,133507638,545,1461393,barang yg dikirim sesuai pesanan,"[-0.04224526137113571, 0.12197387218475342, -0...",1,human
3,"Php, bilang isi ada diseskripsi pas dipesen be...",1,pertukangan,steples tembak / staples tembak kenmaster PROM...,88842566,45,1102298,php bilang isi diseskripsi pa dipesen berbeda ...,"[-0.007697250694036484, 0.055699195712804794, ...",0,bot
4,barang jelek copot&#34; terus tmpat taro isinya,2,pertukangan,Staple Gun / Staples Tembak / Staples Jok / He...,209226141,171,580197,barang jelek copot tmpat taro isinya,"[-0.08997958898544312, 0.18369901180267334, -0...",1,human
...,...,...,...,...,...,...,...,...,...,...,...
1920,Bagus dan memuaskan. Packing bagus cepat,4,pertukangan,Staples Gun Tacker Mollar 3 in 1 / Staple Jok ...,221298191,408,1114588,bagus memuaskan packing bagus cepat,"[-0.03556374832987785, 0.09383474290370941, -0...",1,human
1921,Mantap pengiriman ok cepat recomended,5,pertukangan,Staples Gun Tacker Mollar 3 in 1 / Staple Jok ...,221298191,408,1114588,mantap pengiriman ok cepat recomended,"[-0.07521238178014755, 0.018553180620074272, -...",1,human
1922,Pengiriman cepat. Lumayan,4,pertukangan,Staples Gun Tacker Mollar 3 in 1 / Staple Jok ...,221298191,408,1114588,pengiriman cepat lumayan,"[-0.015180562622845173, -0.004802150186151266,...",1,human
1923,belom dicoba semoga oke barangnya,4,pertukangan,Staples Gun Tacker Mollar 3 in 1 / Staple Jok ...,221298191,408,1114588,belom dicoba semoga oke barangnya,"[-0.01723123900592327, 0.12312690913677216, -0...",0,bot


modelling

In [36]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
from tqdm import tqdm

# Load dataset (adjust this part to your dataset)
data = pd.read_csv('/content/drive/MyDrive/Dataset /data_real.csv')  # Replace with the actual dataset path

# Preprocessing and data preparation
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Load IndoBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('indobenchmark/indobert-base-p2')
model = AutoModelForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p2', num_labels=2)  # Adjust num_labels to your classification task

# Split dataset
# Changed 'text' to 'Ulasan' to match the actual column name in your DataFrame
train_texts, val_texts, train_labels, val_labels = train_test_split(data['Ulasan'], data['label'], test_size=0.2, random_state=42)

# Create datasets and dataloaders
train_dataset = CustomDataset(train_texts.tolist(), train_labels.tolist(), tokenizer, max_length=128)
val_dataset = CustomDataset(val_texts.tolist(), val_labels.tolist(), tokenizer, max_length=128)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Training setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
criterion = torch.nn.CrossEntropyLoss()

def train_epoch(model, data_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    correct_predictions = 0
    for batch in tqdm(data_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        loss = criterion(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct_predictions += (logits.argmax(1) == labels).sum().item()

    return total_loss / len(data_loader), correct_predictions / len(data_loader.dataset)

def eval_model(model, data_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct_predictions = 0
    with torch.no_grad():
        for batch in tqdm(data_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = criterion(logits, labels)

            total_loss += loss.item()
            correct_predictions += (logits.argmax(1) == labels).sum().item()

    return total_loss / len(data_loader), correct_predictions / len(data_loader.dataset)

# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = eval_model(model, val_loader, criterion, device)

    print(f'Epoch {epoch + 1}/{num_epochs}')
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
    print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

# Save the model
model.save_pretrained('indobert_finetuned_model')
tokenizer.save_pretrained('indobert_finetuned_model')

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 25/25 [00:02<00:00,  8.87it/s]


Epoch 1/3
Train Loss: 0.2949, Train Acc: 0.8818
Val Loss: 0.2129, Val Acc: 0.9247


100%|██████████| 25/25 [00:02<00:00,  9.09it/s]


Epoch 2/3
Train Loss: 0.1386, Train Acc: 0.9539
Val Loss: 0.2257, Val Acc: 0.9377


100%|██████████| 25/25 [00:02<00:00,  8.81it/s]


Epoch 3/3
Train Loss: 0.1054, Train Acc: 0.9682
Val Loss: 0.2387, Val Acc: 0.9247


('indobert_finetuned_model/tokenizer_config.json',
 'indobert_finetuned_model/special_tokens_map.json',
 'indobert_finetuned_model/vocab.txt',
 'indobert_finetuned_model/added_tokens.json',
 'indobert_finetuned_model/tokenizer.json')

evaluasi

In [39]:
# Assuming 'model' and 'val_loader' are defined and available from your training code

# Get predictions for the validation dataset
all_predictions = []
with torch.no_grad():
    for batch in tqdm(val_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = logits.argmax(1).cpu().numpy()  # Get predicted labels and move to CPU
        all_predictions.extend(predictions)  # Add predictions to the list

# Convert to NumPy array
y_pred = np.array(all_predictions)

# Now you can calculate the accuracy
accuracy = accuracy_score(val_labels, y_pred)  # Use val_labels as the true labels
print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(classification_report(val_labels, y_pred))

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 25/25 [00:02<00:00,  8.44it/s]

Accuracy: 0.9247
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.94      0.92       185
           1       0.94      0.91      0.93       200

    accuracy                           0.92       385
   macro avg       0.92      0.93      0.92       385
weighted avg       0.93      0.92      0.92       385

